In [69]:
# MES(제조 실행 시스템): 생산 실적, 작업 지시 데이터 -> 생산량, 작업 시간, 생산 일자
# QMS (품질 관리 시스템) : 검사 결과, 불량 이력 => 검사값, 합격/불합격, 불량 유형
# EAM(설비 자산 관리): 설비 정보, 정비 이력 -> 설비명, 제조사, 정비 비용
# IoT 플랫폼: 센서 데이터 -> 온도, 압력, 진동

In [70]:
# 설비별 생산량은 많은데, 품질 불량률도 높은 설비는 어디? -> 생산 + 품질
# 정비 비용이 많이 드는 설비가 실제 생산 기여도가 낮다면? -> 생산 + 정비
# 특정 온도 범위에서 불량률이 증가하는가? -> 센서 + 품질
# 신규 설비와 구형 설비의 생산 효율 차이? -> 설비정보 + 생산 

In [71]:
# 1. concat 함수
# 2. merge 함수(sql의 join)
# 3. join 함수(두 데이터프레임의 인덱스를 먼저 다 맞춰야 한다)

In [72]:
import pandas as pd

In [73]:
production_df = pd.read_csv('./data/05_production.csv', parse_dates=['production_date'])
production_df

,production_id,equipment_id,product_code,production_date,start_time,end_time,target_quantity,actual_quantity,good_quantity,defect_quantity,cycle_time,work_order_no,lot_no,operator_id,shift,created_at,updated_at
0,1,INJ-001,BUMPER-A,2024-01-01,2024-01-01 08:14:00,2024-01-01 09:53:32,97,81,77,4,73.73,WO202401019935,LOT2024010100101,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48
1,2,INJ-001,BUMPER-A,2024-01-01,2024-01-01 21:02:00,2024-01-01 22:33:43,83,78,72,6,70.56,WO202401012535,LOT2024010100110,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48
2,3,INJ-002,BUMPER-A,2024-01-01,2024-01-01 10:12:00,2024-01-01 13:16:28,149,135,132,3,81.99,WO202401018359,LOT2024010100201,OP001,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48
3,4,INJ-002,DASH-C,2024-01-01,2024-01-01 12:48:00,2024-01-01 15:16:31,100,92,90,2,96.87,WO202401016574,LOT2024010100202,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48
4,5,INJ-002,DOOR-B,2024-01-01,2024-01-01 20:48:00,2024-01-01 23:12:13,123,129,122,7,67.08,WO202401012674,LOT2024010100210,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867,1868,PRESS-002,BUMPER-A,2024-03-31,2024-03-31 20:19:00,2024-03-31 23:25:19,150,144,119,25,77.63,WO202403317101,LOT2024033100210,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48
1868,1869,PRESS-002,DASH-C,2024-03-31,2024-04-01 00:15:00,2024-04-01 02:59:58,136,130,109,21,76.15,WO202403318434,LOT2024033100211,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48
1869,1870,PRESS-002,BUMPER-A,2024-03-31,2024-04-01 05:53:00,2024-04-01 07:26:15,84,80,66,14,69.95,WO202403317294,LOT2024033100212,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48
1870,1871,ASM-001,BUMPER-A,2024-03-31,2024-03-31 10:24:00,2024-03-31 13:25:41,143,121,101,20,90.10,WO202403317268,LOT2024033100101,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48


In [74]:
production_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1872 entries, 0 to 1871
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   production_id    1872 non-null   int64         
 1   equipment_id     1872 non-null   object        
 2   product_code     1872 non-null   object        
 3   production_date  1872 non-null   datetime64[ns]
 4   start_time       1872 non-null   object        
 5   end_time         1872 non-null   object        
 6   target_quantity  1872 non-null   int64         
 7   actual_quantity  1872 non-null   int64         
 8   good_quantity    1872 non-null   int64         
 9   defect_quantity  1872 non-null   int64         
 10  cycle_time       1872 non-null   float64       
 11  work_order_no    1872 non-null   object        
 12  lot_no           1872 non-null   object        
 13  operator_id      1872 non-null   object        
 14  shift            1872 non-null   object 

In [75]:
# 1월 생산 데이터와 2월 생산 데이터 분리
df_1 = production_df[production_df['production_date'].dt.month == 1]
df_2 = production_df[production_df['production_date'].dt.month == 2]

In [76]:
df_1.head(2)

,production_id,equipment_id,product_code,production_date,start_time,end_time,target_quantity,actual_quantity,good_quantity,defect_quantity,cycle_time,work_order_no,lot_no,operator_id,shift,created_at,updated_at
0,1,INJ-001,BUMPER-A,2024-01-01,2024-01-01 08:14:00,2024-01-01 09:53:32,97,81,77,4,73.73,WO202401019935,LOT2024010100101,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48
1,2,INJ-001,BUMPER-A,2024-01-01,2024-01-01 21:02:00,2024-01-01 22:33:43,83,78,72,6,70.56,WO202401012535,LOT2024010100110,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48


In [77]:
df_2.head(2)

,production_id,equipment_id,product_code,production_date,start_time,end_time,target_quantity,actual_quantity,good_quantity,defect_quantity,cycle_time,work_order_no,lot_no,operator_id,shift,created_at,updated_at
626,627,INJ-001,DASH-C,2024-02-01,2024-02-01 08:23:00,2024-02-01 10:55:52,128,121,116,5,75.80,WO202402017052,LOT2024020100101,OP010,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48
627,628,INJ-001,BUMPER-A,2024-02-01,2024-02-01 20:16:00,2024-02-01 22:47:50,136,129,120,9,70.62,WO202402018619,LOT2024020100110,OP005,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48


In [78]:
# column이 동일할 때 합치기
df_comb = pd.concat([df_1, df_2], ignore_index=True)
df_comb

,production_id,equipment_id,product_code,production_date,start_time,end_time,target_quantity,actual_quantity,good_quantity,defect_quantity,cycle_time,work_order_no,lot_no,operator_id,shift,created_at,updated_at
0,1,INJ-001,BUMPER-A,2024-01-01,2024-01-01 08:14:00,2024-01-01 09:53:32,97,81,77,4,73.73,WO202401019935,LOT2024010100101,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48
1,2,INJ-001,BUMPER-A,2024-01-01,2024-01-01 21:02:00,2024-01-01 22:33:43,83,78,72,6,70.56,WO202401012535,LOT2024010100110,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48
2,3,INJ-002,BUMPER-A,2024-01-01,2024-01-01 10:12:00,2024-01-01 13:16:28,149,135,132,3,81.99,WO202401018359,LOT2024010100201,OP001,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48
3,4,INJ-002,DASH-C,2024-01-01,2024-01-01 12:48:00,2024-01-01 15:16:31,100,92,90,2,96.87,WO202401016574,LOT2024010100202,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48
4,5,INJ-002,DOOR-B,2024-01-01,2024-01-01 20:48:00,2024-01-01 23:12:13,123,129,122,7,67.08,WO202401012674,LOT2024010100210,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1223,1224,PRESS-002,BUMPER-A,2024-02-29,2024-02-29 22:29:00,2024-03-01 00:43:33,114,109,93,16,74.07,WO202402299728,LOT2024022900210,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48
1224,1225,PRESS-002,DASH-C,2024-02-29,2024-03-01 02:46:00,2024-03-01 04:37:35,87,83,69,14,80.67,WO202402296968,LOT2024022900211,OP005,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48
1225,1226,PRESS-002,DOOR-B,2024-02-29,2024-03-01 04:09:00,2024-03-01 06:05:17,118,113,96,17,61.75,WO202402297721,LOT2024022900212,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48
1226,1227,ASM-001,DASH-C,2024-02-29,2024-02-29 09:17:00,2024-02-29 12:14:22,116,98,84,14,108.59,WO202402295201,LOT2024022900101,OP002,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48


In [79]:
eq_df = pd.read_csv('./data/01_equipment.csv')
eq_df

,equipment_id,equipment_name,equipment_type,location,rated_capacity,installation_date,status,created_at,updated_at
0,INJ-001,사출기 1호기,사출기,A동 1라인,150.0,2020-03-15,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00
1,INJ-002,사출기 2호기,사출기,A동 1라인,150.0,2021-06-20,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00
2,PRESS-001,프레스 1호기,프레스,A동 2라인,200.0,2019-05-10,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00
3,PRESS-002,프레스 2호기,프레스,A동 2라인,200.0,2022-08-25,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00
4,ASM-001,조립라인 1호기,조립라인,B동 1라인,100.0,2020-11-30,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00


In [80]:
pd.merge(production_df, eq_df, on='equipment_id', how='left') # how=['inner', 'left', 'right']

,production_id,equipment_id,product_code,production_date,start_time,end_time,target_quantity,actual_quantity,good_quantity,defect_quantity,...,created_at_x,updated_at_x,equipment_name,equipment_type,location,rated_capacity,installation_date,status,created_at_y,updated_at_y
0,1,INJ-001,BUMPER-A,2024-01-01,2024-01-01 08:14:00,2024-01-01 09:53:32,97,81,77,4,...,2026-01-30 00:42:48,2026-01-30 00:42:48,사출기 1호기,사출기,A동 1라인,150.0,2020-03-15,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00
1,2,INJ-001,BUMPER-A,2024-01-01,2024-01-01 21:02:00,2024-01-01 22:33:43,83,78,72,6,...,2026-01-30 00:42:48,2026-01-30 00:42:48,사출기 1호기,사출기,A동 1라인,150.0,2020-03-15,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00
2,3,INJ-002,BUMPER-A,2024-01-01,2024-01-01 10:12:00,2024-01-01 13:16:28,149,135,132,3,...,2026-01-30 00:42:48,2026-01-30 00:42:48,사출기 2호기,사출기,A동 1라인,150.0,2021-06-20,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00
3,4,INJ-002,DASH-C,2024-01-01,2024-01-01 12:48:00,2024-01-01 15:16:31,100,92,90,2,...,2026-01-30 00:42:48,2026-01-30 00:42:48,사출기 2호기,사출기,A동 1라인,150.0,2021-06-20,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00
4,5,INJ-002,DOOR-B,2024-01-01,2024-01-01 20:48:00,2024-01-01 23:12:13,123,129,122,7,...,2026-01-30 00:42:48,2026-01-30 00:42:48,사출기 2호기,사출기,A동 1라인,150.0,2021-06-20,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867,1868,PRESS-002,BUMPER-A,2024-03-31,2024-03-31 20:19:00,2024-03-31 23:25:19,150,144,119,25,...,2026-01-30 00:42:48,2026-01-30 00:42:48,프레스 2호기,프레스,A동 2라인,200.0,2022-08-25,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00
1868,1869,PRESS-002,DASH-C,2024-03-31,2024-04-01 00:15:00,2024-04-01 02:59:58,136,130,109,21,...,2026-01-30 00:42:48,2026-01-30 00:42:48,프레스 2호기,프레스,A동 2라인,200.0,2022-08-25,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00
1869,1870,PRESS-002,BUMPER-A,2024-03-31,2024-04-01 05:53:00,2024-04-01 07:26:15,84,80,66,14,...,2026-01-30 00:42:48,2026-01-30 00:42:48,프레스 2호기,프레스,A동 2라인,200.0,2022-08-25,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00
1870,1871,ASM-001,BUMPER-A,2024-03-31,2024-03-31 10:24:00,2024-03-31 13:25:41,143,121,101,20,...,2026-01-30 00:42:48,2026-01-30 00:42:48,조립라인 1호기,조립라인,B동 1라인,100.0,2020-11-30,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00


In [81]:
# 생산 데이터 + 설비 정보 데이터
step1 = pd.merge(production_df, eq_df, on='equipment_id', how='left')
step1

,production_id,equipment_id,product_code,production_date,start_time,end_time,target_quantity,actual_quantity,good_quantity,defect_quantity,...,created_at_x,updated_at_x,equipment_name,equipment_type,location,rated_capacity,installation_date,status,created_at_y,updated_at_y
0,1,INJ-001,BUMPER-A,2024-01-01,2024-01-01 08:14:00,2024-01-01 09:53:32,97,81,77,4,...,2026-01-30 00:42:48,2026-01-30 00:42:48,사출기 1호기,사출기,A동 1라인,150.0,2020-03-15,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00
1,2,INJ-001,BUMPER-A,2024-01-01,2024-01-01 21:02:00,2024-01-01 22:33:43,83,78,72,6,...,2026-01-30 00:42:48,2026-01-30 00:42:48,사출기 1호기,사출기,A동 1라인,150.0,2020-03-15,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00
2,3,INJ-002,BUMPER-A,2024-01-01,2024-01-01 10:12:00,2024-01-01 13:16:28,149,135,132,3,...,2026-01-30 00:42:48,2026-01-30 00:42:48,사출기 2호기,사출기,A동 1라인,150.0,2021-06-20,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00
3,4,INJ-002,DASH-C,2024-01-01,2024-01-01 12:48:00,2024-01-01 15:16:31,100,92,90,2,...,2026-01-30 00:42:48,2026-01-30 00:42:48,사출기 2호기,사출기,A동 1라인,150.0,2021-06-20,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00
4,5,INJ-002,DOOR-B,2024-01-01,2024-01-01 20:48:00,2024-01-01 23:12:13,123,129,122,7,...,2026-01-30 00:42:48,2026-01-30 00:42:48,사출기 2호기,사출기,A동 1라인,150.0,2021-06-20,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867,1868,PRESS-002,BUMPER-A,2024-03-31,2024-03-31 20:19:00,2024-03-31 23:25:19,150,144,119,25,...,2026-01-30 00:42:48,2026-01-30 00:42:48,프레스 2호기,프레스,A동 2라인,200.0,2022-08-25,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00
1868,1869,PRESS-002,DASH-C,2024-03-31,2024-04-01 00:15:00,2024-04-01 02:59:58,136,130,109,21,...,2026-01-30 00:42:48,2026-01-30 00:42:48,프레스 2호기,프레스,A동 2라인,200.0,2022-08-25,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00
1869,1870,PRESS-002,BUMPER-A,2024-03-31,2024-04-01 05:53:00,2024-04-01 07:26:15,84,80,66,14,...,2026-01-30 00:42:48,2026-01-30 00:42:48,프레스 2호기,프레스,A동 2라인,200.0,2022-08-25,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00
1870,1871,ASM-001,BUMPER-A,2024-03-31,2024-03-31 10:24:00,2024-03-31 13:25:41,143,121,101,20,...,2026-01-30 00:42:48,2026-01-30 00:42:48,조립라인 1호기,조립라인,B동 1라인,100.0,2020-11-30,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00


In [82]:
quality_df = pd.read_csv('./data/07_quality_inspection.csv')
quality_df

,inspection_id,production_id,equipment_id,product_code,inspection_time,inspection_type,result,defect_code,measurement_value,measurement_unit,inspector_id,lot_no,sample_size,notes,created_at
0,1,1,INJ-001,BUMPER-A,2024-01-01 09:41:18,FINAL,PASS,\N,300.8279,mm,OP007,LOT2024010100101,1,NaN,2026-01-30 01:24:59
1,2,1,INJ-001,BUMPER-A,2024-01-01 08:17:24,FINAL,PASS,\N,299.7696,mm,OP008,LOT2024010100101,1,NaN,2026-01-30 01:24:59
2,3,1,INJ-001,BUMPER-A,2024-01-01 08:47:26,FINAL,PASS,\N,301.0795,mm,OP007,LOT2024010100101,1,NaN,2026-01-30 01:24:59
3,4,1,INJ-001,BUMPER-A,2024-01-01 08:33:03,FINAL,PASS,\N,302.5384,mm,OP007,LOT2024010100101,1,NaN,2026-01-30 01:24:59
4,5,1,INJ-001,BUMPER-A,2024-01-01 09:46:23,FINAL,PASS,\N,299.6097,mm,OP007,LOT2024010100101,1,NaN,2026-01-30 01:24:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37412,37413,1872,ASM-001,DOOR-B,2024-03-31 21:13:26,FINAL,FAIL,D002,608.2289,mm,OP008,LOT2024033100110,1,NaN,2026-01-30 01:25:02
37413,37414,1872,ASM-001,DOOR-B,2024-03-31 20:50:20,FINAL,FAIL,D001,609.7039,mm,OP007,LOT2024033100110,1,NaN,2026-01-30 01:25:02
37414,37415,1872,ASM-001,DOOR-B,2024-03-31 20:23:22,FINAL,FAIL,D010,590.4729,mm,OP008,LOT2024033100110,1,NaN,2026-01-30 01:25:02
37415,37416,1872,ASM-001,DOOR-B,2024-03-31 20:44:08,FINAL,FAIL,D001,609.0846,mm,OP008,LOT2024033100110,1,NaN,2026-01-30 01:25:02


In [83]:
def get_count(x):
    return (x == 'FAIL').sum()

quality_summary = quality_df.groupby('production_id').agg(검사_건수=('inspection_id', 'count'),
                                                          불량_건수=('result', get_count),
                                                          평균_측정값=('measurement_value', 'mean'))
quality_summary

,검사_건수,불량_건수,평균_측정값
production_id,,,
1,11,4,300.583600
2,13,6,299.203377
3,13,3,298.295115
4,11,2,400.704636
5,17,7,598.715941
...,...,...,...
1868,35,25,297.999489
1869,31,21,400.624397
1870,20,14,301.137370


In [84]:
# 생산 + 설비 + 품질데이터
comb = pd.merge(step1, quality_summary, on='production_id')
comb

,production_id,equipment_id,product_code,production_date,start_time,end_time,target_quantity,actual_quantity,good_quantity,defect_quantity,...,equipment_type,location,rated_capacity,installation_date,status,created_at_y,updated_at_y,검사_건수,불량_건수,평균_측정값
0,1,INJ-001,BUMPER-A,2024-01-01,2024-01-01 08:14:00,2024-01-01 09:53:32,97,81,77,4,...,사출기,A동 1라인,150.0,2020-03-15,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,11,4,300.583600
1,2,INJ-001,BUMPER-A,2024-01-01,2024-01-01 21:02:00,2024-01-01 22:33:43,83,78,72,6,...,사출기,A동 1라인,150.0,2020-03-15,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,13,6,299.203377
2,3,INJ-002,BUMPER-A,2024-01-01,2024-01-01 10:12:00,2024-01-01 13:16:28,149,135,132,3,...,사출기,A동 1라인,150.0,2021-06-20,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,13,3,298.295115
3,4,INJ-002,DASH-C,2024-01-01,2024-01-01 12:48:00,2024-01-01 15:16:31,100,92,90,2,...,사출기,A동 1라인,150.0,2021-06-20,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,11,2,400.704636
4,5,INJ-002,DOOR-B,2024-01-01,2024-01-01 20:48:00,2024-01-01 23:12:13,123,129,122,7,...,사출기,A동 1라인,150.0,2021-06-20,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,17,7,598.715941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867,1868,PRESS-002,BUMPER-A,2024-03-31,2024-03-31 20:19:00,2024-03-31 23:25:19,150,144,119,25,...,프레스,A동 2라인,200.0,2022-08-25,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,35,25,297.999489
1868,1869,PRESS-002,DASH-C,2024-03-31,2024-04-01 00:15:00,2024-04-01 02:59:58,136,130,109,21,...,프레스,A동 2라인,200.0,2022-08-25,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,31,21,400.624397
1869,1870,PRESS-002,BUMPER-A,2024-03-31,2024-04-01 05:53:00,2024-04-01 07:26:15,84,80,66,14,...,프레스,A동 2라인,200.0,2022-08-25,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,20,14,301.137370
1870,1871,ASM-001,BUMPER-A,2024-03-31,2024-03-31 10:24:00,2024-03-31 13:25:41,143,121,101,20,...,조립라인,B동 1라인,100.0,2020-11-30,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,30,20,300.775347


In [85]:
comb.to_csv('./data/comb.csv', index=False)

In [86]:
comb = pd.read_csv('./data/comb.csv')
comb

,production_id,equipment_id,product_code,production_date,start_time,end_time,target_quantity,actual_quantity,good_quantity,defect_quantity,...,equipment_type,location,rated_capacity,installation_date,status,created_at_y,updated_at_y,검사_건수,불량_건수,평균_측정값
0,1,INJ-001,BUMPER-A,2024-01-01,2024-01-01 08:14:00,2024-01-01 09:53:32,97,81,77,4,...,사출기,A동 1라인,150.0,2020-03-15,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,11,4,300.583600
1,2,INJ-001,BUMPER-A,2024-01-01,2024-01-01 21:02:00,2024-01-01 22:33:43,83,78,72,6,...,사출기,A동 1라인,150.0,2020-03-15,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,13,6,299.203377
2,3,INJ-002,BUMPER-A,2024-01-01,2024-01-01 10:12:00,2024-01-01 13:16:28,149,135,132,3,...,사출기,A동 1라인,150.0,2021-06-20,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,13,3,298.295115
3,4,INJ-002,DASH-C,2024-01-01,2024-01-01 12:48:00,2024-01-01 15:16:31,100,92,90,2,...,사출기,A동 1라인,150.0,2021-06-20,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,11,2,400.704636
4,5,INJ-002,DOOR-B,2024-01-01,2024-01-01 20:48:00,2024-01-01 23:12:13,123,129,122,7,...,사출기,A동 1라인,150.0,2021-06-20,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,17,7,598.715941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867,1868,PRESS-002,BUMPER-A,2024-03-31,2024-03-31 20:19:00,2024-03-31 23:25:19,150,144,119,25,...,프레스,A동 2라인,200.0,2022-08-25,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,35,25,297.999489
1868,1869,PRESS-002,DASH-C,2024-03-31,2024-04-01 00:15:00,2024-04-01 02:59:58,136,130,109,21,...,프레스,A동 2라인,200.0,2022-08-25,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,31,21,400.624397
1869,1870,PRESS-002,BUMPER-A,2024-03-31,2024-04-01 05:53:00,2024-04-01 07:26:15,84,80,66,14,...,프레스,A동 2라인,200.0,2022-08-25,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,20,14,301.137370
1870,1871,ASM-001,BUMPER-A,2024-03-31,2024-03-31 10:24:00,2024-03-31 13:25:41,143,121,101,20,...,조립라인,B동 1라인,100.0,2020-11-30,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,30,20,300.775347


In [87]:
# 생산 + 설비 + 품질 검사 데이터 -> 설비별 생산 효율, 품질 문제 동시 파악 가능
# 1. 어떤 설비가 생산량은 많은데, 불량률도 높은가?
# 2. 설비 제조사별 품질 차이가 있는가?
# 3. 특정 제품에서 불량이 집중되는지?

In [88]:
# 불량률 계산
comb['불량률'] = (comb['불량_건수']/comb['검사_건수']*100).round(2).fillna(0)
comb

,production_id,equipment_id,product_code,production_date,start_time,end_time,target_quantity,actual_quantity,good_quantity,defect_quantity,...,location,rated_capacity,installation_date,status,created_at_y,updated_at_y,검사_건수,불량_건수,평균_측정값,불량률
0,1,INJ-001,BUMPER-A,2024-01-01,2024-01-01 08:14:00,2024-01-01 09:53:32,97,81,77,4,...,A동 1라인,150.0,2020-03-15,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,11,4,300.583600,36.36
1,2,INJ-001,BUMPER-A,2024-01-01,2024-01-01 21:02:00,2024-01-01 22:33:43,83,78,72,6,...,A동 1라인,150.0,2020-03-15,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,13,6,299.203377,46.15
2,3,INJ-002,BUMPER-A,2024-01-01,2024-01-01 10:12:00,2024-01-01 13:16:28,149,135,132,3,...,A동 1라인,150.0,2021-06-20,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,13,3,298.295115,23.08
3,4,INJ-002,DASH-C,2024-01-01,2024-01-01 12:48:00,2024-01-01 15:16:31,100,92,90,2,...,A동 1라인,150.0,2021-06-20,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,11,2,400.704636,18.18
4,5,INJ-002,DOOR-B,2024-01-01,2024-01-01 20:48:00,2024-01-01 23:12:13,123,129,122,7,...,A동 1라인,150.0,2021-06-20,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,17,7,598.715941,41.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867,1868,PRESS-002,BUMPER-A,2024-03-31,2024-03-31 20:19:00,2024-03-31 23:25:19,150,144,119,25,...,A동 2라인,200.0,2022-08-25,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,35,25,297.999489,71.43
1868,1869,PRESS-002,DASH-C,2024-03-31,2024-04-01 00:15:00,2024-04-01 02:59:58,136,130,109,21,...,A동 2라인,200.0,2022-08-25,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,31,21,400.624397,67.74
1869,1870,PRESS-002,BUMPER-A,2024-03-31,2024-04-01 05:53:00,2024-04-01 07:26:15,84,80,66,14,...,A동 2라인,200.0,2022-08-25,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,20,14,301.137370,70.00
1870,1871,ASM-001,BUMPER-A,2024-03-31,2024-03-31 10:24:00,2024-03-31 13:25:41,143,121,101,20,...,B동 1라인,100.0,2020-11-30,ACTIVE,2024-01-01 00:00:00,2024-01-01 00:00:00,30,20,300.775347,66.67


In [112]:
eq_summary = comb.groupby('equipment_name').agg({'production_id':'count',
                                    'actual_quantity':'sum',
                                    '검사_건수':'sum',
                                    '불량_건수':'sum'})
eq_summary.columns = ['생산건수', '총생산량', '총검사수', '불량건수']
eq_summary

,생산건수,총생산량,총검사수,불량건수
equipment_name,,,,
사출기 1호기,262,28163,5302,3017
사출기 2호기,430,51958,8528,4504
조립라인 1호기,234,22485,4581,2726
프레스 1호기,468,52069,9295,5123
프레스 2호기,478,51929,9711,5506


In [113]:
eq_summary['불량률'] = (eq_summary['불량건수']/eq_summary['총검사수']*100).round(2)
eq_summary

,생산건수,총생산량,총검사수,불량건수,불량률
equipment_name,,,,,
사출기 1호기,262,28163,5302,3017,56.90
사출기 2호기,430,51958,8528,4504,52.81
조립라인 1호기,234,22485,4581,2726,59.51
프레스 1호기,468,52069,9295,5123,55.12
프레스 2호기,478,51929,9711,5506,56.70


In [114]:
# 정비 비용은 많이 들지만 생산 기여도가 낮은 설비는?
main_df = pd.read_csv('./data/10_maintenance_history.csv')
main_df

,equipment_id,maintenance_type,start_time,end_time,maintenance_description,parts_replaced,cost,technician_id,downtime_hours,notes,created_at
0,INJ-002,PREVENTIVE,2024-01-03 10:06:38,2024-01-03 11:38:28,월간 정기점검,NaN,1597990.91,OP015,1.53,다음 정비 시 추가 점검 요망,2026-01-30 02:23:22.393125
1,INJ-002,PREVENTIVE,2024-01-05 15:47:52,2024-01-05 17:47:18,정기 윤활유 교환,"금형, 압력센서",2488706.64,OP011,1.99,NaN,2026-01-30 02:23:22.396196
2,PRESS-002,PREVENTIVE,2024-01-06 21:08:07,2024-01-06 22:49:13,정기 윤활유 교환,금형,1330618.46,OP017,1.69,NaN,2026-01-30 02:23:22.398028
3,ASM-001,PREVENTIVE,2024-01-07 08:00:00,2024-01-07 09:11:21,월간 정기점검,NaN,755786.19,OP017,1.19,NaN,2026-01-30 02:23:22.399504
4,INJ-001,BREAKDOWN,2024-01-08 12:30:24,2024-01-08 14:23:11,설비 고장으로 긴급 수리,"금형, 온도센서",5046550.70,OP013,1.88,NaN,2026-01-30 02:23:22.401063
...,...,...,...,...,...,...,...,...,...,...,...
93,PRESS-001,PREVENTIVE,2024-03-28 09:10:35,2024-03-28 10:12:04,예방 점검 및 청소,금형,1333410.34,OP012,1.02,NaN,2026-01-30 02:23:22.549613
94,INJ-001,PREVENTIVE,2024-03-28 17:29:10,2024-03-28 19:12:37,부품 수명 도달로 인한 교체,NaN,1161144.12,OP012,1.72,정비 완료 후 정상 가동 확인,2026-01-30 02:23:22.548244
95,ASM-001,PREVENTIVE,2024-03-29 16:32:37,2024-03-29 18:12:32,정기 윤활유 교환,볼트체결기,2042709.86,OP011,1.67,NaN,2026-01-30 02:23:22.551113
96,ASM-001,PREVENTIVE,2024-03-31 09:05:30,2024-03-31 10:38:02,월간 정기점검,"그리퍼, 컨베이어벨트",2673073.11,OP020,1.54,NaN,2026-01-30 02:23:22.553850


In [115]:
prod_summary = production_df.groupby('equipment_id').agg({'production_id':'count',
                                           'actual_quantity':'sum'})
prod_summary.columns = ['생산건수', '총생산량']
prod_summary

,생산건수,총생산량
equipment_id,,
ASM-001,234,22485
INJ-001,262,28163
INJ-002,430,51958
PRESS-001,468,52069
PRESS-002,478,51929


In [116]:
main_summary = main_df.groupby('equipment_id').agg({'maintenance_type':'count',
                                                    'cost':'sum',
                                                    'downtime_hours':'sum'}).round(2)
main_summary.columns = ['정비건수', '총정비비용', '총정비시간']
main_summary

,정비건수,총정비비용,총정비시간
equipment_id,,,
ASM-001,19,60072924.01,30.79
INJ-001,17,45291758.85,27.85
INJ-002,23,75076538.50,40.36
PRESS-001,18,44564270.41,29.69
PRESS-002,21,70921381.60,37.71


In [117]:
eq_df2 = eq_df[['equipment_id', 'equipment_name', 'equipment_type']]
eq_df2

,equipment_id,equipment_name,equipment_type
0,INJ-001,사출기 1호기,사출기
1,INJ-002,사출기 2호기,사출기
2,PRESS-001,프레스 1호기,프레스
3,PRESS-002,프레스 2호기,프레스
4,ASM-001,조립라인 1호기,조립라인


In [118]:
eq_analysis = pd.merge(eq_df2, prod_summary, on='equipment_id')
eq_analysis

,equipment_id,equipment_name,equipment_type,생산건수,총생산량
0,INJ-001,사출기 1호기,사출기,262,28163
1,INJ-002,사출기 2호기,사출기,430,51958
2,PRESS-001,프레스 1호기,프레스,468,52069
3,PRESS-002,프레스 2호기,프레스,478,51929
4,ASM-001,조립라인 1호기,조립라인,234,22485


In [119]:
eq_analysis = pd.merge(eq_analysis, main_summary, on='equipment_id')
eq_analysis

,equipment_id,equipment_name,equipment_type,생산건수,총생산량,정비건수,총정비비용,총정비시간
0,INJ-001,사출기 1호기,사출기,262,28163,17,45291758.85,27.85
1,INJ-002,사출기 2호기,사출기,430,51958,23,75076538.50,40.36
2,PRESS-001,프레스 1호기,프레스,468,52069,18,44564270.41,29.69
3,PRESS-002,프레스 2호기,프레스,478,51929,21,70921381.60,37.71
4,ASM-001,조립라인 1호기,조립라인,234,22485,19,60072924.01,30.79


In [120]:
# 분모가 0인 경우, Pandas에서는 inf 또는 -inf로 표기
# -> inf를 0으로 변경
import numpy as np

eq_analysis['생산당_정비비용'] = (eq_analysis['총정비비용']/eq_analysis['총생산량']).round(2).replace([np.inf, -np.inf], 0)
eq_analysis

,equipment_id,equipment_name,equipment_type,생산건수,총생산량,정비건수,총정비비용,총정비시간,생산당_정비비용
0,INJ-001,사출기 1호기,사출기,262,28163,17,45291758.85,27.85,1608.20
1,INJ-002,사출기 2호기,사출기,430,51958,23,75076538.50,40.36,1444.95
2,PRESS-001,프레스 1호기,프레스,468,52069,18,44564270.41,29.69,855.87
3,PRESS-002,프레스 2호기,프레스,478,51929,21,70921381.60,37.71,1365.74
4,ASM-001,조립라인 1호기,조립라인,234,22485,19,60072924.01,30.79,2671.69


In [121]:
eq_analysis = eq_analysis.sort_values('생산당_정비비용', ascending=False)
eq_analysis

,equipment_id,equipment_name,equipment_type,생산건수,총생산량,정비건수,총정비비용,총정비시간,생산당_정비비용
4,ASM-001,조립라인 1호기,조립라인,234,22485,19,60072924.01,30.79,2671.69
0,INJ-001,사출기 1호기,사출기,262,28163,17,45291758.85,27.85,1608.20
1,INJ-002,사출기 2호기,사출기,430,51958,23,75076538.50,40.36,1444.95
3,PRESS-002,프레스 2호기,프레스,478,51929,21,70921381.60,37.71,1365.74
2,PRESS-001,프레스 1호기,프레스,468,52069,18,44564270.41,29.69,855.87


In [123]:
# ROI 문제 설비 찾기: 생산은 적은데 정비비용이 높은 설비
# 총생산량의 중앙값보다 작은 것들을 찾고, 총정비비용의 중앙값보다 높은 것
eq_analysis['총생산량'] < eq_analysis['총생산량'].median()

4     True
0     True
1    False
3    False
2    False
Name: 총생산량, dtype: bool

In [124]:
eq_analysis['총정비비용'] > eq_analysis['총정비비용'].median()

4    False
0    False
1     True
3     True
2    False
Name: 총정비비용, dtype: bool

In [126]:
(eq_analysis['총생산량'] < eq_analysis['총생산량'].median()) & (eq_analysis['총정비비용'] > eq_analysis['총정비비용'].median())

4    False
0    False
1    False
3    False
2    False
dtype: bool